In [66]:
import pandas as pd
import numpy as np
import re
import sys
# options
pd.set_option('display.max_columns', None)

In [67]:
# Load original data
df = pd.read_csv(r'data/fifa21_male2.csv',
                dtype={'Hits': 'object'})
df.head()

,ID,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,Club Logo,Flag Photo,POT,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


In [68]:
# Standardize columns' names and change index
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_').str.replace('&', 'and')
if df.id.is_unique:
    df = df.set_index('id')
df.head()

,name,age,ova,nationality,club,bov,bp,position,player_photo,club_logo,flag_photo,pot,team_and_contract,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,gender
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


#### Data Cleaning
##### First Approach - drop all categorical missing data and sobsitute numerical with mean
##### Explore unicity of values to understand if more data can be dropped

In [69]:
# Explore nan values
orig_rows = df.shape[0]
nan_cols = [col for col in df.columns if df[col].isna().sum() > 0]
nan_counts = df[nan_cols].isna().sum()
nan_dtypes = df[nan_cols].dtypes
df_nan = pd.concat([nan_counts, nan_dtypes], axis=1)
df_nan.columns = ['nan_count', 'dtype']
df_nan

,nan_count,dtype
club,23,object
position,413,object
club_logo,23,object
joined,58,object
loan_date_end,16215,object
volleys,58,float64
curve,58,float64
agility,58,float64
balance,58,float64
jumping,58,float64


In [70]:
# It seems feasible to drop all categorical NaN values except the ones on loan_date_end. 
# The entire column will be dropped due to the high number of NaN values and probable uselessness for our predictions
df = df.drop(['loan_date_end'], axis=1)
nan_cols.remove('loan_date_end')
cat_nan_cols = [col for col in nan_cols if df[col].dtype=='object']  
df = df.dropna(subset=cat_nan_cols)

In [71]:
# replace missing numeric data with the simple average of the column
# a better analysis should at least try to get better values, for example clustering
# the player by their roles (vision and positioning attributes are different if player
# is a defender or attacker)
nan_cols = [col for col in df.columns if df[col].isna().sum() > 0]
df[nan_cols] = df[nan_cols].fillna(df.composure.mean())
print('Dropping missing values has led to a reduction of rows by {} %'.format(
    round((orig_rows-df.shape[0])/orig_rows, 2)
))

Dropping missing values has led to a reduction of rows by 0.03 %


#### EDA starts

##### Study the values count for each columns 
##### Interface this knowledhe with columns.csv description to understand:
#### Which column could be dropped beacuse is insignificant
#### Which column could be dropped sue to its unicity (not apporting any features)
#### Study multicollinearity
#### How to treat outliers


In [72]:
# Explore value counts of our columns
# Write data to a file for accurate exploration
# A one-line description will also be displayed in order to understand column value
df_desc = pd.read_csv('columns.csv', index_col=0)
orig_stdout = sys.stdout
with open('fifa_vc.txt', 'w', encoding='utf-8') as f:
    sys.stdout = f
    for col in df.columns:
        print(df_desc.loc[col].values[0].lstrip())
        print(df[col].value_counts(), end='\n'*3)
    sys.stdout = orig_stdout
    f.close()

In [73]:
# Standardization of some columns with simple methods:
# height --> to SI units (cm)
def to_cm(lst):
    """Converts height from feet, inches to cm"""
    return round(float(lst[0])*30.48+float(lst[1])*2.54, 0)
df.height = df.height.str.findall(r'(\d+)').apply(to_cm).astype('int64')

# weight --> to SI units (kg)
def to_kg(lbs):
    """Converts weight from lbs to kg"""
    lbs = lbs.replace('lbs', '')
    return round(int(lbs), 0)
df.weight = df.weight.apply(to_kg) # force integer

# value --> to euro
def to_float(value):
    """Converts string value to float number"""
    value = value.replace('€', '')
    if 'K' in value:
        return float(value.replace('K', '')) * 1e3
    elif 'M' in value:
        return float(value.replace('M', '')) * 1e6
    else:
        return float(value)
df.value = df.value.apply(to_float)

# wage --> to euro
df.wage = df.wage.apply(to_float)

#release clause --> to euro
df.release_clause = df.release_clause.apply(to_float)

#hits --> to number
df.hits = df.hits.apply(to_float)

# Handling the star statistics
star_stats = ['w/f', 'sm', 'ir']
for col in star_stats:
    df[col] = df[col].str[0].astype('int64')


In [74]:
# Functions for logic on 'contract' columns
def extract_start(contract):
    """Extract start year of contract"""
    if len(contract) == 0:
        return np.nan # no years is found
    elif len(contract) == 2: 
        if int(contract[0])>1900 and int(contract[0])<=int(contract[1]):
            return int(contract[0]) # start year is found
        else:
            return int(contract[1]) # only start year is present
    else:
        # lenght=3
        contract = np.array(contract).astype('int64')
        contract = contract[(contract>2000) & (contract < 2023)] # filter out values
        if len(contract) == 2:
            if int(contract[0])>1900 and int(contract[0])<=int(contract[1]):
                return int(contract[0]) # start year is found
            else:
                return contract[0]+'_verify3'
        elif len(contract) == 1: # only start year is found
            return int(contract[0])

def extract_end(contract):
    """Extract end year of contract"""
    if len(contract) == 0:
        return np.nan # no years is found
    elif len(contract) == 2: 
        if int(contract[0])>1900 and int(contract[0])<=int(contract[1]):
            return int(contract[1]) # end year is found
        else:
            return np.nan # end year is not present
    else:
        # lenght=3
        contract = np.array(contract).astype('int64')
        contract = contract[(contract>2000) & (contract < 2023)] # filter out values
        if len(contract) == 2:
            if int(contract[0])>1900 and int(contract[0])<=int(contract[1]):
                return int(contract[1]) # end year is found
            else:
                return contract[1]+'_verify3'
        elif len(contract) == 1: 
            return np.nan # end year is not present

In [75]:
# Logic for contract columns: range years of contract are extracted in following format:
# contract --> contract_start    contract_end
# contract column will be dropped

# Drop nan values found from contract start and
# replace missing value from contract_end with actual year (meaning contract is not finished)
# Then drop contract column
df.insert(list(df.columns).index('contract'), 'contract_start', 
            df.contract.str.findall(r'(\d+)').apply(extract_start))
df.insert(list(df.columns).index('contract_start'), 'contract_end', 
            df.contract.str.findall(r'(\d+)').apply(extract_end))
df.contract_end = df.contract_end.fillna(2022) # maybe better use datetime library
df = df.dropna()

In [76]:
# Function for logic on range stats
def range_stat(rs):
    """ Returns ranged stat in format 'min_stat, max_stat' """
    values = re.findall(r'\d+', rs)
    if '+-' in rs:
        min_value = str(int(values[0]) - int(values[1]))
    elif '+' in rs and '-' not in rs:
        min_value = values[0]
    max_value = str(int(values[0]) + int(values[1]))
    return min_value + ',' + max_value

In [77]:
# Some statistics are presented in the form:
# value+-range 
# These columns will be separated into a value_min and value_max
# In order to treat them numerically
# Original columns will be dropped
range_stats = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 
                'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 
                'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 
                'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 
                'rcb', 'rb', 'gk']

for col in range_stats:
    df[col] = df[col].apply(range_stat)
    df.insert(list(df.columns).index(col), col+'_min',
        df[col].str.split(',').str[0].astype('int64'))
    df.insert(list(df.columns).index(col), col+'_max',
        df[col].str.split(',').str[1].astype('int64'))
    df = df.drop(col, axis=1)

In [78]:
# The following columns have been identified as useless for 
# my target predictions
cols_to_drop = ['player_photo', 'club_logo', 'flag_photo', 
                'gender', 'contract'] # all players are male
df = df.drop(cols_to_drop, axis=1)
# Now we should have our final dataset
df = df.reset_index(drop=True)
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16332 entries, 0 to 16331
Data columns (total 129 columns):
 #    Column             Non-Null Count  Dtype  
---   ------             --------------  -----  
 0    name               16332 non-null  object 
 1    age                16332 non-null  int64  
 2    ova                16332 non-null  int64  
 3    nationality        16332 non-null  object 
 4    club               16332 non-null  object 
 5    bov                16332 non-null  int64  
 6    bp                 16332 non-null  object 
 7    position           16332 non-null  object 
 8    pot                16332 non-null  int64  
 9    team_and_contract  16332 non-null  object 
 10   height             16332 non-null  int64  
 11   weight             16332 non-null  int64  
 12   foot               16332 non-null  object 
 13   growth             16332 non-null  int64  
 14   joined             16332 non-null  object 
 15   value              16332 non-null  float64
 16   wa

In [79]:
print(*[df[col].value_counts() for col in df.columns], sep='\n'*3)

J. Rodríguez       10
A. Traoré           8
J. Rojas            8
Paulinho            7
J. Sánchez          6
                   ..
M. Kranevitter      1
J. Musso            1
M. Ibáñez           1
L. Miranda          1
Pablo Rodríguez     1
Name: name, Length: 15445, dtype: int64


22    1308
23    1290
21    1256
20    1245
24    1202
25    1095
26    1058
19    1006
28    1004
27     995
29     830
30     715
18     657
31     602
32     488
33     412
34     321
17     288
35     202
36     125
37      73
38      55
16      45
39      25
40      19
41       6
42       6
43       2
47       1
53       1
Name: age, dtype: int64


64    969
67    965
65    963
66    933
68    892
70    868
69    866
63    828
72    770
71    751
62    737
61    618
73    597
75    593
60    564
74    519
59    459
58    391
76    346
57    311
77    310
56    249
78    240
55    223
79    190
54    149
53    144
80    143
52    115
81    104
51     90
83     78
82     69
50     67
84     43
49     40
